# Generate SA scores for qm9 molecules

In [ ]:
import numpy as np
import pandas as pd
from p_tqdm import p_map
from sa_sc_syba_score_parallel import *
import multiprocessing
%matplotlib inline

## Read cleaned qm9 data

In [ ]:
qm9_filepath = '../Computing Emin for QM9/data/clean/qm9.csv'
mol_data = pd.read_csv(qm9_filepath)
os.makedirs('sa_score_sublists',exist_ok=True)

## Define SA score from smiles string function

In [ ]:
def parallelize_dataframe(df,func,n_splits=100,n_cores=multiprocessing.cpu_count()):
    # taken from https://towardsdatascience.com/make-your-own-super-pandas-using-multiproc-1c04f41944a1

    df_split = list(zip(np.array_split(df,n_splits),range(n_splits)))
    p_map(func,df_split,num_cpus = n_cores)

## Calculate SA scores into mol_data dataframe and overwrite cleaned qm9 data file

In [ ]:
parallelize_dataframe(mol_data['smiles_1'],parallel_apply_sa_score)
mol_data['smiles_1 sa_score'] = np.concatenate([np.genfromtxt(f'sa_score_sublists/sa_score_{i}.txt',dtype=str) for i in range(100)])
mol_data.to_csv(qm9_filepath)